### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [11]:
user_mat = np.random.rand(20, 4) # user matrix filled with random values of shape user x latent 
movie_mat = np.random.rand(4, 4)

In [34]:
movie_mat[:,2]

array([ nan,  nan,  nan,  nan])

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [39]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    # sse = sum of squared errors
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i, j] > 0:  
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[i,j] - np.dot(user_mat[i,:], movie_mat[:,j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*error*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*error*user_mat[i, k])
        # print results for iteration
        print(iteration+1 , "|" , sse_accum/num_ratings)
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [40]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=10)
# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 | 56.4944918109
2 | 53.4871615712
3 | 50.3803839465
4 | 47.1795158364
5 | 43.8991999482
6 | 40.56290283
7 | 37.201938008
8 | 33.8539003911
9 | 30.5605269897
10 | 27.3651026068


In [41]:
#Compare the predicted and actual results
#predicted
print(np.dot(user_mat, movie_mat))
#actual
print(ratings_mat)

[[ 4.20171849  3.94829596  4.07806771  4.39003822]
 [ 3.65776231  3.30728147  3.41859168  3.73372579]
 [ 4.12144166  3.47418802  3.98658001  4.2532523 ]
 [ 4.67177727  4.44199374  4.36907698  4.77531364]
 [ 2.98925037  3.03895644  2.93205802  3.17198286]
 [ 3.17150022  2.72918143  3.06076945  3.26525237]
 [ 3.06880893  3.13547922  3.26208699  3.39967753]
 [ 3.94336393  3.29077313  3.94849673  4.15132412]
 [ 4.83742731  4.53215148  4.55763209  4.95866264]
 [ 3.99647578  3.79636611  3.77904435  4.10823165]
 [ 4.2474067   3.74684476  3.95017043  4.29721054]
 [ 3.6862559   3.88095582  3.70209108  3.96016865]
 [ 3.52923482  3.0352433   3.46758819  3.68908023]
 [ 4.64931516  4.13995753  4.35204256  4.71808967]
 [ 3.24932373  2.816965    3.18311935  3.38033055]
 [ 3.03837026  2.64857393  2.96675995  3.16187218]
 [ 3.02333302  3.09264171  3.15539726  3.3119284 ]
 [ 3.86880079  3.96628962  3.96307053  4.20010675]
 [ 3.12007443  2.54544964  2.80964926  3.06792824]
 [ 4.23014422  3.85354372  4.08

**Write your summary here.**

> The results are not very good with 10 iterations.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [42]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 | 44.1118024647
2 | 15.1482709805
3 | 4.11297718975
4 | 2.86942833931
5 | 2.76977341969
6 | 2.73647403268
7 | 2.71114003623
8 | 2.68711155524
9 | 2.66247241891
10 | 2.63630093774
11 | 2.60792174677
12 | 2.57673721795
13 | 2.54219239579
14 | 2.50377355018
15 | 2.46101890821
16 | 2.41353647593
17 | 2.36102659431
18 | 2.30330662411
19 | 2.24033423816
20 | 2.17222493427
21 | 2.09925906374
22 | 2.02187436049
23 | 1.9406419601
24 | 1.85622714385
25 | 1.76933991544
26 | 1.68068388776
27 | 1.59091351336
28 | 1.50060849302
29 | 1.41027020414
30 | 1.32033924584
31 | 1.23122749506
32 | 1.14335424468
33 | 1.05717521046
34 | 0.973195552019
35 | 0.891962672224
36 | 0.81403998314
37 | 0.739967531181
38 | 0.670218197075
39 | 0.60515859615
40 | 0.545021974917
41 | 0.489897090733
42 | 0.439733335002
43 | 0.394359225352
44 | 0.35350950456
45 | 0.316855615367
46 | 0.284035017819
47 | 0.254676191277
48 | 0.228417706939
49 | 0.20492108838
50 | 0.183

In [43]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99999864   9.9999993    9.99999985  10.00000121]
 [ 10.00001616   4.00000656   9.00000102   9.99998679]
 [  7.99999941   8.99999939   9.99999979   5.00000066]
 [  8.99999228   7.99999651   9.99999929  10.00000637]
 [  9.999993     4.99999676   8.99999938   9.00000586]
 [  5.99998702   3.99999468   9.99999914   6.00001067]
 [  9.00000851   8.00000351  10.00000066   8.99999317]
 [  9.99998359   4.99999318   8.99999894   8.00001354]
 [  6.99997461   7.99998963   9.99999856   8.00002104]
 [  8.99998792   4.99999548   8.99999966   7.00001009]
 [  9.00001095   8.00000514  10.0000013    7.99999125]
 [  9.00000935  10.00000414  10.00000102   8.99999262]
 [  9.99999025   8.99999618   9.99999965   8.00000823]
 [  4.99998377   7.99999345   4.99999917   8.00001348]
 [ 10.00001373   8.00000621  10.00000144   9.99998897]
 [  9.00001993   9.00000861  10.00000162   9.99998376]
 [  9.00001043   8.00000447   8.00000069   7.99999137]
 [  9.99999276   7.99999685   0.99999935  10.00000584]
 [  5.0000

**Write your summary here.** <br>
Increasing iterations makes the predicted values a lot better, however it seems the sse_accum is a curve, at certain point, the sse_accum is the minimal, with more increased iterations, the sse_accum is increasing again. <br>

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [44]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [50]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 | 43.2892760361
2 | 15.9215146523
3 | 4.70439810068
4 | 2.91652868833
5 | 2.62837302935
6 | 2.52143440245
7 | 2.44331213789
8 | 2.36639904335
9 | 2.28398954366
10 | 2.19426874366
11 | 2.09698676552
12 | 1.99271341238
13 | 1.88265356238
14 | 1.76855195757
15 | 1.65256480233
16 | 1.53707273939
17 | 1.42445119522
18 | 1.31683737376
19 | 1.21594074165
20 | 1.12293401609
21 | 1.03843836041
22 | 0.962589795743
23 | 0.895154927066
24 | 0.835659167998
25 | 0.783498495864
26 | 0.738019968178
27 | 0.69856973159
28 | 0.664515763642
29 | 0.63525536342
30 | 0.610216069176
31 | 0.588855530116
32 | 0.57066271498
33 | 0.555160639059
34 | 0.541909715058
35 | 0.530510631782
36 | 0.52060595498
37 | 0.511880095619
38 | 0.504057693307
39 | 0.496900726298
40 | 0.490204776586
41 | 0.483794882678
42 | 0.477521348967
43 | 0.471255788499
44 | 0.464887582482
45 | 0.458320860044
46 | 0.451472040979
47 | 0.44426794229
48 | 0.436644422935
49 | 0.42854552589

In [51]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.312359728565168:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [52]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 | 23.1791345428
2 | 10.771593586
3 | 7.4033151639
4 | 5.70482523969
5 | 4.63136213317
6 | 3.88067136773
7 | 3.32345595066
8 | 2.8935015699
9 | 2.5524724834
10 | 2.27624230886
11 | 2.04878684584
12 | 1.85904917563
13 | 1.69915085627
14 | 1.56330509858
15 | 1.44713348596
16 | 1.3472287972
17 | 1.26087329602
18 | 1.1858567069
19 | 1.12035770099
20 | 1.06286481803


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [60]:
np.count_nonzero(first_1000_users), sum(np.isnan(first_1000_users))

(31245000, matrix([[1000, 1000, 1000, ..., 1000, 1000, 1000]]))

In [62]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = sum(np.isnan(first_1000_users)) # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is [[1000 1000 1000 ..., 1000 1000 1000]]


In [63]:
np.isnan(first_1000_users).s

matrix([[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")